# Build Autoencoder

Build an autoencoder with the data provided below as training data.  

For the testing data provided as part of the answer submit a list of indices from the testing data that are anamolies.

In [3]:
!wget https://labs-fannie-data.s3.amazonaws.com/q6.csv

--2019-12-06 17:57:28--  https://labs-fannie-data.s3.amazonaws.com/q6.csv
Resolving labs-fannie-data.s3.amazonaws.com (labs-fannie-data.s3.amazonaws.com)... 52.216.96.3
Connecting to labs-fannie-data.s3.amazonaws.com (labs-fannie-data.s3.amazonaws.com)|52.216.96.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9597176 (9.2M) [text/csv]
Saving to: ‘q6.csv’

q6.csv              100%[===================>]   9.15M  26.2MB/s    in 0.3s    

2019-12-06 17:57:28 (26.2 MB/s) - ‘q6.csv’ saved [9597176/9597176]



In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn import preprocessing

## Construct an autoencoder to detect the outlier

We biuld a deep autoencoder to find the lower dimension representation of the data and try to use the hole model to detect the outlier. If there is a outlier, the result will perform bad interms of the mse.

In [2]:
data = pd.read_csv("q6.csv",header=None)

In [3]:
data2 = preprocessing.scale(data,axis=0)
data.head()

,0,1,2,3,4
0,0.726451,6.149826,1.472649,-1.415477,4.512565
1,-1.915879,5.497435,1.411244,-1.174702,8.800099
2,-0.308676,4.176731,1.455114,-1.510199,6.326681
3,1.089697,5.385993,1.644336,-0.787047,7.538189
4,1.766089,4.868465,1.013386,-1.068426,7.813759


In [4]:
def construct_encoder_decoder(data_):
    inputs = tf.keras.Input(shape=(5,))
    encoder_0 = tf.keras.layers.Dense(units=128, activation='tanh')(inputs)
    encoder_1 = tf.keras.layers.Dense(units=32, activation='tanh')(encoder_0)
    encoder_2 = tf.keras.layers.Dense(units=3, activation='tanh')(encoder_1)
    Encoder = tf.keras.Model(
        inputs=inputs,
        outputs=encoder_2,
        name="encoder"
    )

    decoder_0 = tf.keras.layers.Dense(units=32, activation='tanh')(encoder_2)
    decoder_1 = tf.keras.layers.Dense(units=128, activation='tanh')(decoder_0)
    decoded_last = tf.keras.layers.Dense(units=5, activation='tanh')(decoder_1)
    model = tf.keras.Model(
        inputs=inputs, 
        outputs=decoded_last,
        name='entire_model'
    )
    return model, Encoder




In [5]:
entire, encoder = construct_encoder_decoder(data2)

In [6]:
entire.compile(
    optimizer='adam',
    loss='mae',
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

Fit the data and get the model.

In [7]:
entire.fit(
    data2, data2,
    epochs=100, batch_size=256,
)

Train on 101000 samples
Epoch 1/100
101000/101000 [==============================] - 1s 10us/sample - loss: 0.4513 - root_mean_squared_error: 0.7322
Epoch 2/100
101000/101000 [==============================] - 1s 5us/sample - loss: 0.4293 - root_mean_squared_error: 0.7185
Epoch 3/100
101000/101000 [==============================] - 1s 5us/sample - loss: 0.4267 - root_mean_squared_error: 0.7169
Epoch 4/100
101000/101000 [==============================] - 1s 5us/sample - loss: 0.4225 - root_mean_squared_error: 0.7150
Epoch 5/100
101000/101000 [==============================] - 1s 5us/sample - loss: 0.4190 - root_mean_squared_error: 0.7137
Epoch 6/100
101000/101000 [==============================] - 1s 5us/sample - loss: 0.4153 - root_mean_squared_error: 0.7116
Epoch 7/100
101000/101000 [==============================] - 1s 5us/sample - loss: 0.4135 - root_mean_squared_error: 0.7101
Epoch 8/100
101000/101000 [==============================] - 1s 5us/sample - loss: 0.4120 - root_mean_squar

101000/101000 [==============================] - 1s 5us/sample - loss: 0.4033 - root_mean_squared_error: 0.7005
Epoch 67/100
101000/101000 [==============================] - 1s 5us/sample - loss: 0.4034 - root_mean_squared_error: 0.7005
Epoch 68/100
101000/101000 [==============================] - 1s 5us/sample - loss: 0.4032 - root_mean_squared_error: 0.7005
Epoch 69/100
101000/101000 [==============================] - 1s 5us/sample - loss: 0.4032 - root_mean_squared_error: 0.7005
Epoch 70/100
101000/101000 [==============================] - 1s 5us/sample - loss: 0.4032 - root_mean_squared_error: 0.7005
Epoch 71/100
101000/101000 [==============================] - 1s 5us/sample - loss: 0.4030 - root_mean_squared_error: 0.7004
Epoch 72/100
101000/101000 [==============================] - 1s 5us/sample - loss: 0.4033 - root_mean_squared_error: 0.7004
Epoch 73/100
101000/101000 [==============================] - 1s 5us/sample - loss: 0.4029 - root_mean_squared_error: 0.7004
Epoch 74/100


Generate the data and compare the predicted data with the original data to see the root mean square error of each data point. This rmse cna be seen as a distance between the two data and we find the 95% quantile of this distance and let it to be the threshold and give the index of those data that extends this threshold.

In [16]:
data_hat = entire.predict(data2)

In [17]:
diff = (data_hat-data2)**2

In [18]:
mse = np.sqrt(diff.mean(axis=1))

In [26]:
threshold = np.quantile(mse,0.95)

In [33]:
result_index = data.index[mse>=threshold]
print(result_index[:100])

Int64Index([  18,   40,   48,   64,   73,   95,   98,  133,  164,  176,  186,
             192,  194,  209,  218,  233,  247,  261,  297,  353,  356,  381,
             397,  403,  436,  466,  500,  504,  526,  537,  574,  581,  599,
             607,  622,  649,  658,  670,  698,  707,  711,  727,  731,  747,
             807,  828,  835,  852,  856,  871,  900,  921,  972,  998,  999,
            1020, 1042, 1044, 1063, 1064, 1066, 1073, 1076, 1102, 1147, 1160,
            1189, 1212, 1215, 1216, 1250, 1253, 1254, 1256, 1313, 1358, 1359,
            1367, 1389, 1391, 1472, 1488, 1491, 1493, 1494, 1506, 1522, 1537,
            1572, 1625, 1653, 1667, 1709, 1729, 1734, 1796, 1822, 1841, 1860,
            1872],
           dtype='int64')


In [36]:
result_index = list(result_index)

In [37]:
import json
with open('outlier_index.txt', 'w') as filehandle:
    json.dump(result_index, filehandle)